# Import google oauth python script to setup api

In [13]:
from Google import Create_Service

# Define pertinent variables for service call

In [14]:
CLIENT_SECRET_FILE = 'key.json'
DRIVE_API_NAME = 'drive'
SHEETS_API_NAME = 'sheets'
DRIVE_API_VERSION = 'v3'
SHEETS_API_VERSION = 'v4'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
FOLDER_NAME = 'Job Stuff'
FILE_NAME = "Scraper Job Tracker Sheet"

# Call create service for Google Drive API

In [15]:
service = Create_Service(CLIENT_SECRET_FILE, DRIVE_API_NAME, DRIVE_API_VERSION, SCOPES)

key.json-drive-v3-(['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
drive service created successfully


# Request a certain folder, if not found, create the folder

In [16]:
response = service.files().list(q="name = '" + FOLDER_NAME + "' and mimeType = 'application/vnd.google-apps.folder' and trashed=false", spaces='drive', fields='nextPageToken, files(id, name)').execute()
folderId = None
if(response['files']):
    folderId = response['files'][0]['id']
else:
    file_metadata = {
        'name': FOLDER_NAME,
        'mimeType': 'application/vnd.google-apps.folder',
    }
    folder = service.files().create(body=file_metadata, fields='id').execute()
    folderid = folder.get('id')

# Find spreadsheet, if it doesnt exist, make the sheet

In [17]:
response = service.files().list(q="parents='" + folderId + "' and name = '" + FILE_NAME + "' and mimeType = 'application/vnd.google-apps.spreadsheet' and trashed=false", spaces='drive', fields='nextPageToken, files(id, name)').execute()
spreadsheetId = None
isCreated = False
worksheetId = None
if(response['files']):
    spreadsheetId = response['files'][0]['id']
else:
    isCreated = True
    file_metadata = {
        'name': FILE_NAME,
        'mimeType': 'application/vnd.google-apps.spreadsheet',
        'parents': [folderId]
    }
    sheet = service.files().create(body=file_metadata, fields='id').execute()
    spreadsheetId = sheet.get('id')

# Call Create Service for Google Sheets API

In [18]:
service = Create_Service(CLIENT_SECRET_FILE, SHEETS_API_NAME, SHEETS_API_VERSION, SCOPES)

key.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
sheets service created successfully


# Format sheet header and worksheet if just created

### Rename worksheet name

In [19]:
jblisting = "Job Listing"
if(isCreated):
    response = service.spreadsheets().get(spreadsheetId=spreadsheetId).execute()
    worksheetId = response['sheets'][0]['properties']['sheetId']
    requests = []
    requests.append({
        'updateSheetProperties': {
            'properties': {
                'sheetId': worksheetId,
                'title': jblisting
            },
            'fields': 'title'
        }
    })
    body = {
        'requests': requests
    }
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheetId,
        body = body
    ).execute()
else:
    response = service.spreadsheets().get(spreadsheetId=spreadsheetId).execute()
    worksheetId = response['sheets'][0]['properties']['sheetId']   


### Insert header text

In [20]:
if(isCreated):
    values = [
        [
        'Job Title',
        'Company',
        'Post Date',
        'Date Applied',
        'City,Zip',
        'Salary',
        'Job Type',
        'Number of Hires',
        'Benefits',
        'Schedule',
        'Education',
        'Experience',
        'Remote',
        'Work Location',
        'Interview',
        'Follow-up Contact'
        ]
    ]
    body = {
        'values': values
    }
    response = service.spreadsheets().values().update(
        spreadsheetId = spreadsheetId, range=jblisting+'!A1', valueInputOption='USER_ENTERED', body=body
    ).execute()

### Format Header

In [21]:
if(isCreated):
    requests = []
    requests.append(
    {
        "repeatCell":{
            "range":{
            "sheetId": worksheetId,
            "startColumnIndex": 0,
            "endColumnIndex": 16,
            "startRowIndex": 0,
            "endRowIndex": 1
        },
        #rgb(183,183,183)
            "cell": {
                "userEnteredFormat":{
                    "backgroundColor":{
                        "red": 0.7177,
                        "green": 0.7177,
                        "blue": 0.7177
                    },
                    "horizontalAlignment": "CENTER",
                    "textFormat":{
                        "foregroundColor":{
                            "red": 1.0,
                            "green": 1.0,
                            "blue": 1.0
                        },
                        "fontSize" : 10
                    }
                }
            },
            "fields": "userEnteredFormat(backgroundColor, textFormat, horizontalAlignment)"
        }
    })
    requests.append({
        "updateSheetProperties": {
            "properties": {
                "sheetId": worksheetId,
                "gridProperties": {
                    "frozenRowCount": 1
                }
            },
            "fields": "gridProperties.frozenRowCount"
        }
    })
    requests.append({
        "autoResizeDimensions": {
            "dimensions": {
                "sheetId": worksheetId,
                "dimension": "COLUMNS",
                "startIndex": 0,
                "endIndex": 16
            }
        }
    })
    body = {
        "requests": requests
    }
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheetId, body=body
    ).execute()

# Format the data cells

### Center the boxes horizontal and set text font

In [24]:
if(isCreated):
    requests = []
    requests.append({
        "repeatCell":{
            "range":{
            "sheetId": worksheetId,
            "startColumnIndex": 0,
            "endColumnIndex": 16,
        },
            "cell": {
                "userEnteredFormat":{
                    "horizontalAlignment": "CENTER",
                    "textFormat":{
                        "fontSize" : 10
                    }
                }
            },
            "fields": "userEnteredFormat(textFormat, horizontalAlignment)"
        }
    })

### Set some cells to auto resize

In [ ]:
if(isCreated):
    requests.append({
        "autoResizeDimensions": {
            "dimensions": {
                "sheetId": worksheetId,
                "dimension": "COLUMNS",
                "startIndex": 0,
                "endIndex": 6
            }
        }
    })
    requests.append({
        "autoResizeDimensions": {
            "dimensions": {
                "sheetId": worksheetId,
                "dimension": "COLUMNS",
                "startIndex": 13,
                "endIndex": 16
            }
        }
    })

### Expand width of all cells

In [ ]:
if(isCreated):
    requests.append({
        "updateDimensionProperties":{
            "range":{
                "sheetId": worksheetId,
                "dimension":"COLUMNS",
                "startIndex": 0,
                "endIndex": 16
            },
            "properties":{
                "pixelSize": 150
            },
            "fields": "pixelSize"
        }
    })

### If info is not listed, or anything is set to no, set the cells color to red.

In [ ]:
if(isCreated):
    requests.append({
        "addConditionalFormatRule":{
            "rule":{
                "ranges":[
                    {
                        "sheetId": worksheetId,
                        "startColumnIndex": 0,
                        "endColumnIndex": 16,
                        "startRowIndex": 1
                    }
                ],
                "booleanRule":{
                    "condition":{
                        "type": "TEXT_EQ",
                        "values":[
                            {
                                "userEnteredValue": "No",
                                "userEnteredValue": "Not Listed"
                            }
                        ]
                    },
                    "format":{
                        "backgroundColor": {
                            "red": 1.0,
                            "green": 0,
                            "blue": 0
                        },
                        "textFormat":{
                            "foregroundColor":{
                                "red": 1.0,
                                "green": 1.0,
                                "blue": 1.0
                            }
                        }
                    }
                }
            }
        }
    })

### Reset blanks cells color to white

In [ ]:
if(isCreated):
    requests.append({
        "addConditionalFormatRule":{
            "rule":{
                "ranges":[
                    {
                        "sheetId": worksheetId,
                        "startColumnIndex": 0,
                        "endColumnIndex": 16,
                        "startRowIndex": 1
                    }
                ],
                "booleanRule":{
                    "condition":{
                        "type": "BLANK",
                    },
                    "format":{
                        "backgroundColor": {
                            "red": 1.0,
                            "green": 1.0,
                            "blue": 1.0
                        }
                    }
                }
            }
        }
    })

### If cells are equal to yes, the cells color is green.

In [ ]:
if(isCreated):
    requests.append({
        "addConditionalFormatRule":{
            "rule":{
                "ranges":[
                    {
                        "sheetId": worksheetId,
                        "startColumnIndex": 0,
                        "endColumnIndex": 16,
                        "startRowIndex": 1
                    }
                ],
                "booleanRule":{
                    "condition":{
                        "type": "TEXT_EQ",
                        "values":[
                            {
                                "userEnteredValue": "Yes",
                            }
                        ]
                    },
                    "format":{
                        "backgroundColor": {
                            "red": 0,
                            "green": 1.0,
                            "blue": 0
                        },
                        "textFormat":{
                            "foregroundColor":{
                                "red": 1.0,
                                "green": 1.0,
                                "blue": 1.0
                            }
                        }
                    }
                }
            }
        }
    })

### Set text to wrap and set list for follow-up email and interview

In [26]:
if(isCreated):
    requests.append({
        "repeatCell":{
            "range":{
                "sheetId": worksheetId,
                "startColumnIndex": 0,
                "endColumnIndex": 16,
                "startRowIndex": 1
            },
            "cell":{
                "userEnteredFormat":{
                    "wrapStrategy": "WRAP",
                    "verticalAlignment": "TOP"
            }
            },
            "fields": "userEnteredFormat(wrapStrategy,verticalAlignment)"
        }
    })
    requests.append({
        "repeatCell":{
            "range":{
                "sheetId": worksheetId,
                "startColumnIndex": 14,
                "endColumnIndex": 16,
                "startRowIndex": 1
            },
            "cell":{
                "dataValidation":{
                    "condition":{
                        "type": "ONE_OF_LIST",
                        "values": [
                            {
                                "userEnteredValue": "Yes",
                            },
                            {
                                "userEnteredValue": "No"
                            }
                        ]
                    },
                    "strict": True,
                    "showCustomUi": True
                }
            },
            "fields": "dataValidation(condition, strict, showCustomUi)"
        }
    })

### Send in batchupdate of forms

In [ ]:
if(isCreated):
    body = {
        "requests": requests
    }
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheetId, body=body
    ).execute()

# Insert form data from job listing
## We are using mock data

In [23]:
#['Lab Assistant-Office Help', 'Turf Tech, Inc.', '08/132021', '08/14/2021', 'Tangent, OR 97389', 'From $12 an hour', 'Full-time, Part-time', '3', 'Not Listed', '8 hour shift, Monday to Friday, On call', "Bachelor's (Preferred)", 'Not Listed', 'No', 'One location', 'No', 'No']
values = [
    [
        'Lab Assistant-Office Help', 
        'Turf Tech, Inc.', 
        '08/132021', '08/14/2021', 
        'Tangent, OR 97389', 
        'From $12 an hour', 
        'Full-time, Part-time', 
        '3', 'Not Listed', 
        '8 hour shift, Monday to Friday, On call', 
        "Bachelor's (Preferred)", 
        'Not Listed', 
        'No', 
        'One location', 
        'No', 
        'No'
    ]
]
body = {
    'values': values
}
response = service.spreadsheets().values().append(
    spreadsheetId=spreadsheetId,
    range=jblisting+"!A1",
    valueInputOption="USER_ENTERED",
    body=body
).execute()